In [3]:
from collections import defaultdict
from surprise import Dataset
import pandas as pd
from surprise import SVD
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
import numpy as np
from surprise import dump
import os

import io  # needed because of weird encoding of u.item file

from surprise import KNNBaseline
from surprise import get_dataset_dir



In [ ]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def train_and_save_prediction_model (file_to_save) :
    #train and save model

    df = pd.read_csv('testForInput.csv')
    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(1, 5))

    # The columns must correspond to user id, item id and ratings (in that order).
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
    trainset = data.build_full_trainset()
    #train data with algorithm
    algo = SVD()
    #cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    algo.fit(trainset)
    # Dump algorithm and reload it.
    file_name = os.path.expanduser(file_to_save)
    dump.dump(file_name, algo=algo)

def train_and_save_similarity_model(file_to_save):
    df = pd.read_csv('testForInput.csv')
    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(1, 5))

    # The columns must correspond to user id, item id and ratings (in that order).
    data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
    trainset = data.build_full_trainset()
    sim_options = {'name': 'cosine', 'user_based': False} # or item based
    algo = KNNBaseline(sim_options=sim_options)
    algo.fit(trainset)
    
    # Dump algorithm and reload it.
    file_name = os.path.expanduser(file_to_save)
    dump.dump(file_name, algo=algo)
    
    
    
    
def load_model_from_disk(file_to_load):
    file_name = os.path.expanduser(file_to_load) # 'SVD_dump_file'
    _, loaded_algo = dump.load(file_name)
    return loaded_algo

In [3]:
train_and_save_prediction_model ('SVD_dump_file')

In [39]:
train_and_save_similarity_model('KNNbaseline_dump_file')

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [3]:
prediction_algo = load_model_from_disk('SVD_dump_file')

In [40]:
knn_algo = load_model_from_disk('KNNbaseline_dump_file')

In [ ]:
from flask import Flask
app = Flask(__name__)
df = pd.read_csv('testForInput.csv')
#'ALOXOO497B4LH'
@app.route('/topN/<user_name>')
def get_TopN_items_prediction(user_name):
    
    test = df[df['userID'] == user_name].drop(df.columns[3], axis=1)
    columnsTitles=["userID","itemID","rating"]
    test=test.reindex(columns=columnsTitles)
    predictions = prediction_algo.test(np.array(test))

    top_n = get_top_n(predictions, n=10)
    ret = []
    # Print the recommended items for each user
    for uid, user_ratings in top_n.items():
        ret.append ( (uid, [iid for (iid, _) in user_ratings]) )
    return str(ret[0])    

@app.route('/knn/<item_id>')
def get_k_nearest_neighbour_by_itme (item_id):
    toy_k_neighbors = knn_algo.get_neighbors(int(item_id), k=10)
    return ','.join(str(e) for e in toy_k_neighbors)
    
    


if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Jun/2020 11:31:24] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [14/Jun/2020 11:31:29] "GET /knn/4 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 11:31:32] "GET /knn/5 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 11:31:34] "GET /knn/3 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 11:31:53] "GET /knn/2003 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 11:31:55] "GET /knn/2002 HTTP/1.1" 200 -
127.0.0.1 - - [14/Jun/2020 11:32:01] "GET /knn/2001 HTTP/1.1" 200 -


In [ ]:
from surprise import KNNBasic

df = pd.read_csv('testForInput.csv')
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[['userID', 'itemID', 'rating']], reader)
trainset = data.build_full_trainset()
sim_options = {'name': 'cosine', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

In [ ]:
for i in range ( 2000 , 3000) :
    print (algo.get_neighbors(i, k=10))